In [1]:
import scipy.io
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential # type: ignore
from tensorflow.keras.layers import Dense, LSTM, Dropout, BatchNormalization, Bidirectional, Input, Reshape # type: ignore
from tensorflow.keras.optimizers import Adam, RMSprop  # type: ignore
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau # type: ignore
from tensorflow.keras.regularizers import l2 # type: ignore


from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
print(f"TensorFlow Version: {tf.__version__}")

# Load the .mat file
file_path = '../EV_Rank_1_52_RBs_50_UEs_1000_snaps.mat'
data = scipy.io.loadmat(file_path)

# Extract the relevant data
EV_data = data['EV_re_im_split']
data = EV_data
del EV_data
print(data.shape)

2024-05-28 12:30:14.130285: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow Version: 2.16.1
(50, 1000, 832)


In [2]:
# Function to create sequences
def create_sequences(data, n_steps_in, n_steps_out):
    X, y = [], []
    for i in range(data.shape[0]):  # iterate over samples
        for j in range(data.shape[1] - n_steps_in - n_steps_out + 1):  # iterate over timesteps
            seq_x = data[i, j:j + n_steps_in]
            seq_y = data[i, j + n_steps_in:j + n_steps_in + n_steps_out]
            X.append(seq_x)
            y.append(seq_y)
    return np.array(X), np.array(y)


timesteps_in = 10

X, y = create_sequences(data, timesteps_in,5)
print(f'X shape: {X.shape}, y shape: {y.shape}')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)



X shape: (49300, 10, 832), y shape: (49300, 5, 832)


# Preprocessing

In [3]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((39440, 10, 832), (9860, 10, 832), (39440, 5, 832), (9860, 5, 832))

In [4]:
# Define a function to create the advanced LSTM model with gradient clipping
def create_model(dropout_rate=0.2, lstm_units=256, dense_units=512):
    model = Sequential()
    model.add(Input(shape=(timesteps_in, 832)))
    model.add(Bidirectional(LSTM(lstm_units, return_sequences=True)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Bidirectional(LSTM(lstm_units)))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(dense_units, activation='relu'))
    model.add(Dropout(dropout_rate))
    model.add(BatchNormalization())
    model.add(Dense(832*5))
    model.add(Reshape((5,832)))
    
    optimizer = Adam(clipvalue=1.0)

    model.compile(optimizer=optimizer, loss='mse')
    return model

In [5]:

best_params = {

    'dropout_rate': 0.2,
    'lstm_units': 512,
    'dense_units': 512,
    'batch_size': 512,
    'epochs': 100,
}

# Create and train the best model
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

model = create_model(
    dropout_rate=best_params['dropout_rate'],
    lstm_units=best_params['lstm_units'],
    dense_units=best_params['dense_units'],
)



2024-05-28 12:30:20.264223: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-28 12:30:20.273360: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-28 12:30:20.275715: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:

history = model.fit(
    X_train, y_train,
    batch_size=best_params['batch_size'],
    epochs=best_params['epochs'],
    validation_split=0.2,
    callbacks=[early_stopping, reduce_lr],
    verbose=1)

2024-05-28 12:30:22.993859: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1050050560 exceeds 10% of free system memory.
2024-05-28 12:30:24.880641: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 525025280 exceeds 10% of free system memory.
2024-05-28 12:30:25.299368: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1050050560 exceeds 10% of free system memory.
2024-05-28 12:30:26.078526: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 525025280 exceeds 10% of free system memory.


Epoch 1/100


2024-05-28 12:30:30.600631: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907
2024-05-28 12:30:42.548792: W external/local_tsl/tsl/framework/bfc_allocator.cc:487] Allocator (GPU_0_bfc) ran out of memory trying to allocate 121.01MiB (rounded to 126883840)requested by op StatefulPartitionedCall/gradient_tape/sequential_1/bidirectional_1_2/backward_lstm_1_1/CudnnRNNBackpropV3
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-05-28 12:30:42.548876: I external/local_tsl/tsl/framework/bfc_allocator.cc:1044] BFCAllocator dump for GPU_0_bfc
2024-05-28 12:30:42.548906: I external/local_tsl/tsl/framework/bfc_allocator.cc:1051] Bin (256): 	Total Chunks: 70, Chunks in use: 62. 17.5KiB allocated for chunks. 15.5KiB in use in bin. 300B client-requested in use in bin.
2024-05-28 12:30:42.548928: I extern

InternalError: Graph execution error:

Detected at node gradient_tape/sequential_1/bidirectional_1_2/backward_lstm_1_1/CudnnRNNBackpropV3 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/home/rithvik/anaconda3/lib/python3.11/site-packages/ipykernel_launcher.py", line 17, in <module>

  File "/home/rithvik/anaconda3/lib/python3.11/site-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/home/rithvik/anaconda3/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 701, in start

  File "/home/rithvik/anaconda3/lib/python3.11/site-packages/tornado/platform/asyncio.py", line 195, in start

  File "/home/rithvik/anaconda3/lib/python3.11/asyncio/base_events.py", line 607, in run_forever

  File "/home/rithvik/anaconda3/lib/python3.11/asyncio/base_events.py", line 1922, in _run_once

  File "/home/rithvik/anaconda3/lib/python3.11/asyncio/events.py", line 80,

In [ ]:
test_loss = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Loss: {test_loss}")

predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
r2 = r2_score(y_test, predictions)

print(f'Mean Squared Error (MSE): {mse}')
print(f'Mean Absolute Error (MAE): {mae}')
print(f'R-squared (R^2): {r2}')